In [15]:
"""
参数回归-模型1

介绍：
假定比特币价格基于某个均值函数M(x)进行周期性波动
均值函数M(x)是递增凹函数，一阶导数大于零，二阶导数小于零。
波动周期为正弦函数S(x)。
波动幅值为A(x)，同样是个递增凹函数，并且增幅小于M(x)。
A(x)*S(x)称为波动函数。
实际币价为均值函数与波动函数的叠加。

目标函数: Price(x) = M(x) + A(x) * S(x)
M(x)为均值函数，A(x)为幅值函数，S(x)为周期函数。

1. M(x)=l*(x+x0)^k + c
参数：
1.1 L-(L>0)均值增长的速率
1.2 x0-(x0>0)增长曲线的起始
1.3 k-(0<k<1)控制增速衰减速度，值越小增速下降越快。
1.4 c-(c>=0)控制均值增长的起点

2. A(x)=a * (x+x1)^b + d
参数：
2.1 a-(a>0)控制幅值增长的速率
2.2 x1-(x1>0)幅值函数的起始值
2.3 b-(0<b<1)确保增速逐渐加快（凸性），但增幅弱于M(x)
2.4 d-(d>=0)控制幅值增长的起点

3. S(x)=sin(x/T + x2)
参数：
3.1 T-(T>0)周期
3.2 x2-(0<=x2<=2pi)初始相位

4.另有限制条件：
4.1 a<L
4.2 x1<x0
4.3 b<k
"""

'\n参数回归-模型1\n\n介绍：\n假定比特币价格基于某个均值函数M(x)进行周期性波动\n均值函数M(x)是递增凹函数，一阶导数大于零，二阶导数小于零。\n波动周期为正弦函数S(x)。\n波动幅值为A(x)，同样是个递增凹函数，并且增幅小于M(x)。\nA(x)*S(x)称为波动函数。\n实际币价为均值函数与波动函数的叠加。\n\n目标函数: Price(x) = M(x) + A(x) * S(x)\nM(x)为均值函数，A(x)为幅值函数，S(x)为周期函数。\n\n1. M(x)=l*(x+x0)^k + c\n参数：\n1.1 L-(L>0)均值增长的速率\n1.2 x0-(x0>0)增长曲线的起始\n1.3 k-(0<k<1)控制增速衰减速度，值越小增速下降越快。\n1.4 c-(c>=0)控制均值增长的起点\n\n2. A(x)=a * (x+x1)^b + d\n参数：\n2.1 a-(a>0)控制幅值增长的速率\n2.2 x1-(x1>0)幅值函数的起始值\n2.3 b-(0<b<1)确保增速逐渐加快（凸性），但增幅弱于M(x)\n2.4 d-(d>=0)控制幅值增长的起点\n\n3. S(x)=sin(x/T + x2)\n参数：\n3.1 T-(T>0)周期\n3.2 x2-(0<=x2<=2pi)初始相位\n\n4.另有限制条件：\n4.1 a<L\n4.2 x1<x0\n4.3 b<k\n'

In [47]:
# 导入包和数据
import math

import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from lmfit import Model

# -------------------------------
# 1. 加载数据
# 假设CSV文件 'btc_prices.csv' 中包含 Date 和 Close 两列
df = pd.read_csv('btc_prices.csv', parse_dates=['open_date'], index_col='open_date')
df = df.sort_index()  # 确保按日期排序

# 将日期转换为数值（天数）用于回归分析
df['Time'] = np.arange(len(df))

x_data = df['Time']
y_data = df['open_price']
y_data_log = np.log(df['open_price'])

In [48]:
# 模型和参数
def fn_m(x, l:float, k:float, x0:float, c:float):
    # return l/(1+np.exp(-k*(x-x0)))
    return l*((x+x0)**k) + c

def fn_a(x, a:float, d:float, b:float, x1:float):
    return a * (x+ x1)**b + d

def fn_s(x, x2: float, t:float):
    return np.sin(x/t + x2)

def fn_price(x, a:float, b:float, c:float, d:float, l:float, k:float, x0:float, x1:float, x2:float, t:float):
    return fn_m(x, l, k, x0, c) + fn_a(x, a, d, b, x1) * fn_s(x, x2, t)

def fn_log_price(x, a:float, b:float, c:float, d:float, l:float, k:float, x0:float, x1:float, x2:float, t:float):
    prices = fn_price(x, a, b, c, d, l, k, x0, x1, x2, t)
    # for price in prices:
    #     if price <= 0.0:
    #         print("price<=0 - price:", price, "\tx:", x, "\ta:", a, "\tb:", b, "\tc:", c, "\td:", d, "\tl:", l, "\tk:", k, "\tx0:", x0, "\tx1:", x1, "\tx2:", x2, "\tt:", t)

    # 将小于0的值都置为一个非常小的正数
    prices[prices < 0] = 1e-9
    return np.log(prices)

# 手动提供的参数
INIT_A = 300
INIT_B = 0.6
INIT_C = 1
INIT_D = 0
INIT_L = 600
INIT_K = 0.58
INIT_X0 = 1
INIT_X1 = 600
INIT_X2 = math.pi * 0.5
INIT_T = 1400/math.pi/2

# # 回归获得的参数
# INIT_A = 5.68868564
# INIT_B = 0.99999000
# INIT_C = 291.096217
# INIT_D = 3967.90537
# INIT_L = 18.5985387
# INIT_K = 0.99999000
# INIT_X0 = 11.8565495
# INIT_X1 = 6.24777991
# INIT_X2 = 1.21494460
# INIT_T = 208.538659


# 创建 lmfit 模型
model = Model(fn_price)
model_log = Model(fn_log_price)

# 根据先验知识给出初始猜测，若无先验则可均设为1
# params = model.make_params(a=INIT_A, b=INIT_B, c=INIT_C, d=INIT_D, l=INIT_L, k=INIT_K, x0=INIT_X0, x1=INIT_X1, x2=INIT_X2, t=INIT_T)
params = model_log.make_params(a=INIT_A, b=INIT_B, c=INIT_C, d=INIT_D, l=INIT_L, k=INIT_K, x0=INIT_X0, x1=INIT_X1, x2=INIT_X2, t=INIT_T)

# 如果有参数边界、固定值或者其他约束，可使用：
params['a'].set(min=0+1e-5) # a>0
params['b'].set(min=0+1e-5, max=1-1e-5) # 0<b<1
params['c'].set(min=0+1e-5) # c>0
params['d'].set(min=0) # d>=0
params['l'].set(min=0+1e-5) # L>0
params['k'].set(min=0+1e-5, max=1-1e-5) # 0<k<1
params['x0'].set(min=0+1e-5) # x0>0
params['x1'].set(min=0) # 0<=x1
params['x2'].set(min=0, max=2*math.pi) # 0<=x1<=2pi
params['t'].set(min=0) # T>0


# 准备数据
true_params = [INIT_A, INIT_B, INIT_C, INIT_D, INIT_L, INIT_K, INIT_X0, INIT_X1, INIT_X2, INIT_T]
y_true = fn_price(x_data, *true_params) # 初始参数曲线
y_true_log = fn_log_price(x_data, *true_params) # 初始参数log曲线
y_true_m = fn_m(x_data, l=INIT_L, k=INIT_K, x0=INIT_X0, c=INIT_C) # 均值曲线

In [49]:
# 拟合数据
result = model_log.fit(y_data_log, params, x=x_data, method='trust-constr')
y_regression_fit = np.exp(result.best_fit)

# 输出拟合报告
print(result.fit_report())

# 最终参数
params = result.best_values

#
y_regression_fit_m = fn_m(x_data, l=params['l'], k=params['k'], x0=params['x0'], c=params['c'])
y_regression_fit_m_log = np.log(y_regression_fit_m)

[[Model]]
    Model(fn_log_price)
[[Fit Statistics]]
    # fitting method   = equality_constrained_sqp
    # function evals   = 12254
    # data points      = 2743
    # variables        = 10
    chi-square         = 349.111286
    reduced chi-square = 0.12773922
    Akaike info crit   = -5634.46614
    Bayesian info crit = -5575.29806
    R-squared          = 0.85029954
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    a:   5.41744331 (init = 300)
    b:   0.99999000 (init = 0.6)
    c:   265.279568 (init = 1)
    d:   0.00000000 (init = 0)
    l:   18.7189000 (init = 600)
    k:   0.99999000 (init = 0.58)
    x0:  6.96419606 (init = 1)
    x1:  782.530790 (init = 600)
    x2:  1.21094389 (init = 1.570796)
    t:   208.611129 (init = 222.8169)

In [40]:
# 创建log图表
fig = make_subplots()

# 添加log价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_data_log, mode='lines', name='比特币log价格'))

# 添加初始参数拟合曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true_log, mode='lines', name='初始参数log拟合曲线'))

# 添加初始参数均值曲线
fig.add_trace(go.Scatter(x=df.index, y=np.log(y_true_m), mode='lines', name='初始参数log均值曲线'))

# 添加拟合价格曲线
fig.add_trace(go.Scatter(x=df.index, y=result.best_fit, mode='lines', name='拟合log价格'))

# 添加拟合log均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit_m_log, mode='lines', name='拟合log均值曲线'))

# 更新布局
fig.update_layout(
    title='比特币拟合log价格',
    xaxis_title='日期',
    yaxis_title='价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()

In [41]:
# 创建价格图表
fig = make_subplots()

# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_data, mode='lines', name='比特币价格'))

# 添加初始参数拟合曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true, mode='lines', name='初始参数拟合曲线'))

# 添加初始参数均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true_m, mode='lines', name='初始参数均值曲线'))

# 添加拟合价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit, mode='lines', name='拟合价格'))

# 添加拟合log均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit_m, mode='lines', name='拟合均值曲线'))

# 更新布局
fig.update_layout(
    title='比特币拟合价格',
    xaxis_title='日期',
    yaxis_title='价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()